<a href="https://colab.research.google.com/github/chefPony/genai-lab/blob/master/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install datasets
%pip install transformers
%pip install evaluate
%pip install genaibook
%pip install --upgrade numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.3.3 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.3 which is incompatible.
tensorflow 2.19.0 requires n

In [ ]:
%pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 11.7 MB/s eta 0:00:00


In [ ]:
import numpy as np

## Simple conceptual example

In [ ]:
def quantize(vector, alpha):
  qvector = np.round(vector * alpha)
  return qvector

def dequantize(qvector, alpha):
  return qvector/alpha

vector = np.random.random(size=(1, 10)) * 20 - 10
alpha = 127/np.max(vector)
qvector = quantize(vector, alpha)
dqvector = dequantize(qvector, alpha)

print(f"Scaling Factor: {alpha}")
print(f"Vector: {vector}")
print(f"Quantized Vector: {qvector}")
print(f"Dequantized Vector: {dqvector}")
print(f"Difference: {vector - dqvector}")

Scaling Factor: 18.247927585170263
Vector: [[ 1.11521933  6.21994274  6.95969443 -7.15661483 -9.08261198 -7.85663546
  -8.04683796  4.02991709  0.49511303  3.71700933]]
Quantized Vector: [[  20.  114.  127. -131. -166. -143. -147.   74.    9.   68.]]
Dequantized Vector: [[ 1.09601487  6.24728477  6.95969443 -7.17889741 -9.09692343 -7.83650633
  -8.0557093   4.05525502  0.49320669  3.72645056]]
Difference: [[ 1.92044585e-02 -2.73420280e-02  8.88178420e-16  2.22825720e-02
   1.43114516e-02 -2.01291279e-02  8.87133759e-03 -2.53379309e-02
   1.90633604e-03 -9.44123487e-03]]


## Model Quantization

In [1]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from transformers import pipeline, AutoTokenizer
import torch

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.3",
                                             quantization_config=quantization_config,
                                             device_map="auto")

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

### Training

In [ ]:
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

sft_config = SFTConfig(
    "quantized_peft_fine_tuning",
    push_to_hub=False,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    num_train_epochs=2,
    #eval_strategy="steps",
    #eval_steps=200,
    logging_steps=200,
    gradient_checkpointing=True,
    max_length=256,
    dataset_text_field="text",
    packing=True,
    report_to="none"
)

trainer = SFTTrainer(
    model,
    args=sft_config,
    train_dataset=dataset.select(range(300)),
    peft_config=peft_config
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Packing train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


Step,Training Loss


TrainOutput(global_step=68, training_loss=1.250456641702091, metrics={'train_runtime': 3102.995, 'train_samples_per_second': 0.173, 'train_steps_per_second': 0.022, 'total_flos': 5754694306037760.0, 'train_loss': 1.250456641702091, 'entropy': 1.2679332538562662, 'num_tokens': 134760.0, 'mean_token_accuracy': 0.7017085727523354, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ChefPony/quantized_peft_fine_tuning/commit/9084928a3583927938f00174390ed5d30e4c971e', commit_message='End of training', commit_description='', oid='9084928a3583927938f00174390ed5d30e4c971e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ChefPony/quantized_peft_fine_tuning', endpoint='https://huggingface.co', repo_type='model', repo_id='ChefPony/quantized_peft_fine_tuning'), pr_revision=None, pr_num=None)

## Inference

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.3",
    dtype=torch.float16,
    device_map="auto",
  )


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model

NameError: name 'model' is not defined

In [4]:
from accelerate import disk_offload

In [10]:
del model

NameError: name 'model' is not defined

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "ChefPony/quantized_peft_fine_tuning",
    dtype=torch.float16,
    device_map="auto",
    offload_folder="./offload",
    offload_state_dict=True
    #offload_dir="."
  )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for model.layers.6.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for model.layers.6.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for model.layers.6.self_attn.v_proj.lora_A.default.weight: cop

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: model.norm, model.rotary_emb, lm_head, model.layers.

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
pipe("### Human: Hello!### Assistant:", max_new_tokens=100)